In [1]:
import os
import tweepy
from dotenv import load_dotenv

import json

#### Cargar llaves de la API
La función `load_dotenv` lee el archivo `.env` e inivializa las variables de ambiente que allí están. Luego se puede acceder a dichas variables de ambiente con el módulo `os`

In [2]:
import requests


load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True,
  return_type=requests.Response,
)

La siguiente función recibe las listas de cuentas y tweets estraídos inicialmente para ir agregando las cuentas y los tweets.

In [9]:
def extraer_cuentas_usuarios(iteracion, lista_cuentas, lista_tweets, diccionario_tweets, response):
    #diccionario_tweets.update(response)
    #print(response['includes'])
    for i in range(len(response['includes']['users'])):
        diccionario_tweets['data'][iteracion + i] = response['data'][i]
        diccionario_tweets['includes'][iteracion + i] = response['includes']['users'][i]
        lista_cuentas.append(response['includes']['users'][i])
        lista_tweets.append(response['data'][i]['text'])

    print(len(lista_cuentas))
    return len(diccionario_tweets['data'])


En esta parte es donde se usa el query para buscar Tweets, y luego usa la función anterior para extraer los Tweets que se encuentran en los responses de la paginación que hace Tweepy

In [10]:
query = '-RT (deprimido lang:es -is:retweet deprimida OR desmotivado OR desmotivada OR llorar OR nervioso OR nerviosa OR preocupado OR preocupada OR triste OR vacío OR vacía OR agobiado OR agobiado OR agotado OR agotada OR angustiado OR angustiada OR ansiedad OR ansioso OR ansiosa OR cansado OR cansada OR decaído OR depresión OR depresivo OR depresiva OR desesperado OR desesperada OR insomnio)'

In [11]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days


#query = 'depresión yo diagnosticado'
#query = 'depresión yo tengo'
#query = 'depresión yo tengo -lang:es #depresión #ansiedad'
#query = 'Petro'
#query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
#query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
#diccionario_tweets = {'data':{}, 'includes':{}, 'meta':{}}
diccionario_tweets = {'data':{}, 'includes':{}}
cont = 0
for response in tweepy.Paginator(client.search_all_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                start_time='2021-01-20T00:00:00Z',
                                end_time='2022-01-20T00:00:00Z',
                                max_results=100, limit=100):#.flatten(limit=100):
    cont = extraer_cuentas_usuarios(cont, cuentas_usuarios, tweets, diccionario_tweets, response)
    #print(response)



95


Rate limit exceeded. Sleeping for 756 seconds.


191
289
388
488


Rate limit exceeded. Sleeping for 899 seconds.


587
686
785
883
982
1080


...

9294
9391
9489


Rate limit exceeded. Sleeping for 899 seconds.


9585


Rate limit exceeded. Sleeping for 900 seconds.


9682
9781


In [12]:
with open("tweets.json", "w") as fp:
    json.dump(diccionario_tweets,fp)

In [13]:
print(len(cuentas_usuarios))
#print(cuentas_usuarios)

9781


In [14]:
print(len(diccionario_tweets['data'].keys()))

9781


In [15]:
print(cuentas_usuarios[3])

{'username': 'patlevesque00', 'name': 'patrick levesque', 'id': '1445142983179898889'}


In [16]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    usuarios.append(usuario['username'])
    ids_usuarios.append(usuario['id'])

Crear el diccionario con el id que identifica a la persona y su correspondiente usuario

In [17]:
diccionario_ids_usuarios = dict(zip(ids_usuarios, usuarios))

#### Extraer tweets de usuarios
Con se extraen los tweets de los usuarios que se conservaron (las personas de Colombia). Sin embargo, se crea esta función previa ya que al momento de extraer retweets, cuando estos son muy largos (más de 240 caracteres) queda cortado el mensaje, por lo que realicé esta función, que va verificando si es retweet y en caso afirmativo extrae su texto completo que se encuenta la parte de `includes`

In [16]:
def extraer_tweets_usuarios(id, diccionario, response):
    if id in list(diccionario.keys()):
        lista_tweets = diccionario[id]
    else:
        lista_tweets = []

    cont = 0
    for i in range(len(response.data)):
        if response.data[i]['referenced_tweets']:
            try:
                lista_tweets.append(response.includes['tweets'][cont].text)
                cont += 1
            except:
                lista_tweets.append(response.data[i].text)

        else:
            lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [17]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text', 'created_at'],
                                    user_fields=['id', 'username', 'location', 'public_metrics'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        #extraer_tweets_usuarios(id_usuario, users_tweets, response)
        #print(response.data[0].created_at)
        users_tweets.update({diccionario_ids_usuarios[id_usuario]: response})


In [40]:
with open("users_and_tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

#### Guardar usuarios y Tweets en archivos

In [38]:
with open("tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

In [39]:
with open("users.json", "w") as fp:
    json.dump(diccionario_ids_usuarios,fp)